In [1]:
from ngsolve import *
from ngstrefftz import *
from netgen.geom2d import unit_square
from netgen.csg import unit_cube

importing NGSolve-6.2.2105-204-gb9f5842ea


# embedded Trefftz for Helmholtz
Standard polynomial Trefftz functions for the Helmholtz equation do not exist, to circumvent this problem, we weaken our condition in the Trefftz space. We introduce a projection $\Pi$ that is yet to be defined and define the weak Trefftz space and the embedded weak Trefftz DG method:
$$
\newcommand{\Th}{{\mathcal{T}_h}} 
\newcommand{\Fh}{\mathcal{F}_h} 
\newcommand{\dom}{\Omega} 
\newcommand{\jump}[1]{[\![ #1 ]\!]}
\newcommand{\tjump}[1]{[\![{#1} ]\!]_\tau}
\newcommand{\avg}[1]{\{\!\!\{#1\}\!\!\}}
\newcommand{\nx}{n_\mathbf{x}} 
\newcommand{\Vhp}{V^p(\Th)}
\newcommand{\bT}{\mathbf{T}}
\newcommand{\bW}{\mathbf{W}}
\newcommand{\bw}{\mathbf{w}}
\newcommand{\bl}{\mathbf{l}}
\newcommand{\bM}{\mathbf{M}}
\newcommand{\bL}{\mathbf{L}}
\newcommand{\bA}{\mathbf{A}}
\newcommand{\bU}{\mathbf{U}}
\newcommand{\bV}{\mathbf{V}}
\newcommand{\calL}{\mathcal{L}}
\newcommand{\bu}{\mathbf{u}}
\newcommand{\IT}{\mathbb{T}}
\newcommand{\calG}{\mathcal{G}}
\newcommand{\be}{\mathbf{e}}
\newcommand{\bx}{{\mathbf x}}
\newcommand{\inner}[1]{\langle #1 \rangle}
\begin{align}
    \text{Find }u_{\IT}\in \IT^p(\Th)&,~\text{ s.t. }
    a_h(u_{\IT},v_{\IT})=\ell(v_{\IT})\qquad \forall v_{\IT}\in \IT^p(\Th)\quad \text{ with } \\
    \IT^p(\Th)&=\{v\in \Vhp,\ \Pi \calL v=0\}. \label{eq:weakTspace}
    % \\ &\IT^p(\Th)=\{v\in L^2(\dom) \sst \restr{v}{K}\in\IT(K),\forall K\in\Th\}
\end{align}
$$

This way, we can re-define the matrix $\bW$ for the general case as 
$$\begin{align} \label{def:W3}
    (\bW)_{ij}&=\inner{\calL\phi_j,\tilde\calL\phi_i}_{0,h}. 
\end{align}$$


For the Helmholtz equation with Robin boundary conditions
$$\begin{align*}
    \begin{cases}
    -\Delta u - \omega^2 u= 0 &\text{ in } \dom, \\
    \frac{\partial u}{\partial \nx} + i u = g &\text{ on } \partial \dom.
    \end{cases}
\end{align*}$$

we choose the operators
$$\begin{align*}
\calL=-\Delta u -\omega^2 u && \tilde\calL=-\Delta u
\end{align*}$$

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=.3))
fes = L2(mesh, order=4, complex=True, dgjumps=True)
u,v = fes.TnT()

In [3]:
omega=1
n = specialcf.normal(2)
exact = exp(1j*sqrt(0.5)*(x+y))
gradexact = CoefficientFunction((sqrt(0.5)*1j*exact, sqrt(0.5)*1j*exact))
bndc = gradexact*n + 1j*omega*exact
eps = 10**-7

In [4]:
h = specialcf.mesh_size
alpha = 1/(omega*h)
beta = omega*h
delta = omega*h

jump_u = (u-u.Other())*n
jump_v = (v-v.Other())*n
jump_du = (grad(u)-grad(u.Other()))*n
jump_dv = (grad(v)-grad(v.Other()))*n
mean_u = 0.5 * ((u)+(u.Other()))
mean_du = 0.5 * (grad(u)+grad(u.Other()))
mean_dv = 0.5 * (grad(v)+grad(v.Other()))

a = BilinearForm(fes)
a += grad(u)*(grad(v))*dx - omega**2*u*(v)*dx

a += -(jump_u*(mean_dv)+mean_du*(jump_v)) * dx(skeleton=True)
a += -1/(omega*1j)*beta*(jump_du*(jump_dv)) * dx(skeleton=True)
a += omega*1j*alpha*jump_u*(jump_v) * dx(skeleton=True)

a += -delta*(u*(grad(v))*n+grad(u)*n*(v)) * ds(skeleton=True)
a += -1/(omega*1j)*delta*(grad(u)*n)*((grad(v))*n) * ds(skeleton=True)
a += omega*1j*(1-delta)*u*(v) * ds(skeleton=True)

f = LinearForm(fes)
f += -1/(omega*1j)*delta*bndc*(grad(v))*n*ds(skeleton=True)
f += (1-delta)*bndc*(v)*ds(skeleton=True)

with TaskManager():
    a.Assemble()
    f.Assemble()

In [6]:
Lap = lambda u : sum(Trace(u.Operator('hesse')))
op = (-Lap(u)-u)*(Lap(v))*dx+1j*(-Lap(u)-u)*(Lap(v))*dx
eps = 10**-8
with TaskManager():
    PP = TrefftzEmbedding(op,fes,eps)
PPT = PP.CreateTranspose()
with TaskManager():
    TA = PPT@a.mat@PP
    TU = TA.Inverse()*(PPT*f.vec)
    tpgfu = GridFunction(fes)
    tpgfu.vec.data = PP*TU
error = sqrt(Integrate((tpgfu-exact)*Conj(tpgfu-exact), mesh).real)
print("error ",error)

error  2.897248343079839e-08
